In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from pathlib import Path
from typing import Tuple

In [9]:
def _CYME(df: pd.DataFrame) -> float:
    """ Compute the CYME metric, that is 1/2(median(yearly error) + median(monthly error))"""

    yearly_agg = df.groupby("cluster_nl")[["target", "prediction"]].sum().reset_index()
    yearly_error = abs((yearly_agg["target"] - yearly_agg["prediction"])/yearly_agg["target"]).median()

    monthly_error = abs((df["target"] - df["prediction"])/df["target"]).median()

    return 1/2*(yearly_error + monthly_error)


def _metric(df: pd.DataFrame) -> float:
    """Compute metric of submission.

    :param df: Dataframe with target and 'prediction', and identifiers.
    :return: Performance metric
    """
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"])

    # Split 0 actuals - rest
    zeros = df[df["zero_actuals"] == 1]
    recent = df[df["zero_actuals"] == 0]

    # weight for each group
    zeros_weight = len(zeros)/len(df)
    recent_weight = 1 - zeros_weight

    # Compute CYME for each group
    return round(recent_weight*_CYME(recent) + zeros_weight*min(1,_CYME(zeros)),8)


def compute_metric(submission: pd.DataFrame) -> Tuple[float, float]:
    """Compute metric.

    :param submission: Prediction. Requires columns: ['cluster_nl', 'date', 'target', 'prediction']
    :return: Performance metric.
    """

    submission["date"] = pd.to_datetime(submission["date"])
    submission = submission[['cluster_nl', 'date', 'target', 'prediction', 'zero_actuals']]

    return _metric(submission)

In [10]:
# Define file paths

features_cols = [
    "brand",
    "che_pc_usd",
    "che_perc_gdp",
    "corporation",
    "country",
    "launch_date",
    "ind_launch_date",
    "indication",
    "insurance_perc_che",
    "population",
    "prev_perc",
    "price_month",
    "price_unit",
    "public_perc_che",
    "therapeutic_area",
    "Country_Group",
    "Price_Group",
    "indication_number",
    "avg_price_per_year",
    "month_number"
]
target_col = "target"
id_col = ["date","cluster_nl"]

base_dir = os.path.join(os.path.dirname(os.getcwd()), "dataset")
therap = os.path.join(os.path.dirname(os.getcwd()), "dataset", "therapeutic_area")
# Load datasets
# data = pd.read_csv(f"{base_dir}/train_data.csv", usecols=features_cols + [target_col] + id_col)
data = {}
y = {}
filenames = os.listdir(therap)

therapeutic_areas = []
extracted_parts = []
for filename in filenames:
    filename_no_ext = filename.split('.')[0]
    therapeutic_areas.append(filename_no_ext)
    # Load the dataset
    filepath = f"{therap}/{filename}"
    df = pd.read_csv(filepath, usecols=features_cols + [target_col] + id_col)
    print(f"Data loaded for therapeutic areas: {filename}")
    data[filename_no_ext] = df
    y[filename_no_ext] = df[target_col]
    extracted_parts.append(filename_no_ext)


test_data = pd.read_csv(f"{base_dir}/submission_data_TRY2.csv", usecols=features_cols + id_col)



Data loaded for therapeutic areas: 22ED.csv
Data loaded for therapeutic areas: 96D7.csv
Data loaded for therapeutic areas: 032C_051D_644A_66C5_6CEE.csv
Data loaded for therapeutic areas: 4BA5_645F_8E53_980E.csv
Data loaded for therapeutic areas: CD59.csv


In [11]:
numeric_features = {}
categorical_features = {}
for i in extracted_parts:
    # convert int64 to float64
    data[i] = data[i].astype({"Country_Group": "float64"})
    data[i] = data[i].astype({"Price_Group": "float64"})
    data[i] = data[i].astype({"indication_number": "float64"})
    numeric_features[i] = data[i].select_dtypes(include=['float64']).drop(columns=[target_col], errors='ignore').columns
    categorical_features[i] = data[i].select_dtypes(include=['object']).columns

test_data = test_data.astype({"Country_Group": "float64"})
test_data = test_data.astype({"Price_Group": "float64"})
test_data = test_data.astype({"indication_number": "float64"})

# Separate numeric and categorical features for imputation


In [12]:
X = {}
for i in extracted_parts:
    # Drop unnecessary columns
    X[i] = data[i].drop(columns=[target_col]+["cluster_nl"])

X_test = test_data.drop(columns=["cluster_nl"])

# Preprocessing pipeline
def preprocess_data(X, preprocessor=None, fit=True):
    numerical_features = X.select_dtypes(include=['float64']).columns
    categorical_features = X.select_dtypes(include=['category']).columns

    if preprocessor is None:
        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

    if fit:
        X_transformed = preprocessor.fit_transform(X)
    else:
        X_transformed = preprocessor.transform(X)

    X_transformed = np.array(X_transformed)

    return X_transformed, preprocessor

# Preprocess data
X_transformed = {}
for i in extracted_parts:
    X_transformed[i], preprocessor = preprocess_data(X[i], fit=True)


X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)

In [13]:
X_train = {}
X_valid = {}
y_train = {}
y_valid = {}
for i in extracted_parts:
    print(f"X_transformed shape for {i}: {X_transformed[i].shape}")
    X_train[i], X_valid[i], y_train[i], y_valid[i] = train_test_split(X_transformed[i], y[i], test_size=0.2, shuffle=False)


X_transformed shape for 22ED: (112, 11)
X_transformed shape for 96D7: (45858, 12)
X_transformed shape for 032C_051D_644A_66C5_6CEE: (44331, 12)
X_transformed shape for 4BA5_645F_8E53_980E: (24150, 12)
X_transformed shape for CD59: (3992, 12)


In [ ]:
# Let's train different models for different therapeutic areas
models = {}
for i in extracted_parts:
    models[f'RandomForest-{i}'] =  RandomForestRegressor(n_estimators=100, random_state=42)

print(models)
results = {}
for i in extracted_parts:
    results[i] = 0
for name, model in models.items():
    i = name.split('-')[1]
    model.fit(X_train[i], y_train[i])
    scores = cross_val_score(model, X_train[i], y_train[i], scoring='neg_root_mean_squared_error', cv=5)
    results[name.split('-')[1].split('.')[0]] = -scores.mean()

In [ ]:
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE']).sort_values(by='RMSE')
results_df['RMSE'] = results_df['RMSE'].map("{:.5f}".format)
results_df

In [ ]:
X_valid_df = {}
for i in extracted_parts:
    X_valid_df[i] = pd.DataFrame(X_valid[i])

# Predict on validation set
predictions = {}
for i in extracted_parts:
    predictions[i] = models[f'RandomForest-{i}'].predict(X_valid[i])

# Compute metric
metrics = {}
for i in extracted_parts:
    metrics[i] = compute_metric(pd.concat([X_valid_df[i], pd.Series(y_valid[i]), pd.Series(predictions[i])], axis=1))


    


In [ ]:
X_test_transformed = preprocess_data(X_test, preprocessor=preprocessor, fit=False)
print(X_test_transformed[0])

# Ensure indices align between test_data and X_test_transformed
X_test_transformed = pd.DataFrame(X_test_transformed[0], index=test_data.index)  # Specify column names if necessary

def extract_part(text):
    return text.split('_')[2].split('.')[0]

X_test_transformed['therapeutic_area'] = test_data['therapeutic_area'].apply(extract_part)
print(X_test_transformed)

# Initialize a DataFrame to store the results
results_final = pd.DataFrame()

# Iterate over each therapeutic area
for area_country in models.keys():
    
    keyy = area_country
    
    print(keyy)
    # Filter test data for the current therapeutic area
    area_indices = X_test_transformed[X_test_transformed['therapeutic_area'].isin(keyy)].index
    X_area = X_test_transformed.loc[area_indices].drop('therapeutic_area', axis=1)

    # Get the corresponding model
    model = models[area_country]
    print(models)

    # Make predictions
    if not X_area.empty:
      predictions = model.predict(X_area)

      # Prepare the result DataFrame
      test_area = test_data.loc[area_indices]
      area_results = pd.DataFrame({
          'date': pd.to_datetime(test_area['date']).dt.strftime("%m/%d/%Y"),
          'cluster_nl': test_area['cluster_nl'],
          'prediction': np.exp(predictions)
      })

      # Append to the results
      results_final = pd.concat([results_final, area_results], ignore_index=True)

results_final.to_csv('result.csv', index=False)
print("Result file saved as 'result.csv'")